# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


In [4]:
data['CustomerID'].sort_values().unique()

array([   33,   200,   264,   356,   412,   464,   477,   639,   649,
         669,   694,   756,   883,   891,  1008,  1034,  1066,  1072,
        1336,  1428,  1435,  1534,  1577,  1594,  1754,  1839,  1920,
        2187,  2329,  2503,  2556,  2566,  2582,  2617,  2686,  2754,
        2776,  2902,  2915,  2939,  3074,  3253,  3267,  3305,  3317,
        3472,  3531,  3535,  3544,  3885,  3903,  3909,  4261,  4479,
        4595,  4644,  4723,  5104,  5177,  5183,  5224,  5230,  5434,
        5678,  5739,  5832,  5921,  5968,  5986,  5997,  6001,  6185,
        6317,  6352,  6367,  6447,  6543,  6734,  6817,  6851,  7021,
        7134,  7223,  7543,  7606,  7623,  7772,  7827,  7987,  7999,
        8316,  8367,  8372,  8539,  8711,  8917,  8962,  9047,  9132,
        9260,  9302,  9533,  9546,  9597,  9771,  9804, 10016, 10094,
       10114, 10224, 10311, 10418, 10739, 11023, 11057, 11083, 11102,
       11253, 11325, 11380, 11441, 11443, 11614, 11628, 11744, 11883,
       11933, 12059,

## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [5]:
total_quantity = data.groupby(['CustomerID','ProductName']).agg({'Quantity':'sum'})
total_quantity

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1
...                                            ...
98200      Vol Au Vents                         50
           Wasabi Powder                        25
           Wine - Fume Blanc Fetzer             25
           Wine - Hardys Bankside Shiraz        25
           Yogurt - French Vanilla              25

[63628 rows x 1 columns]

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [6]:
# total_quantity

customer_matrix = pd.pivot_table(total_quantity, index='ProductName',columns='CustomerID', values='Quantity')
customer_matrix = customer_matrix.fillna(0)
customer_matrix

# pandas.pivot_table(data, values=None, index=None, columns=None, aggfunc='mean', fill_value=None, margins=False, dropna=True, margins_name='All', observed=_NoDefault.no_default, sort=True)[source]


CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0
Yoghurt Tubes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0
"Yogurt - Blueberry, 175 Gr",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [7]:
dist_calculation = pdist(X=customer_matrix.T, metric='euclidean')
len(dist_calculation)

499500

In [8]:
# Squareform method: https://scipy.github.io/devdocs/reference/generated/scipy.spatial.distance.squareform.html

dist_distribution = squareform(dist_calculation)
dist_distribution

array([[  0.        ,  11.91637529,  10.48808848, ..., 228.62851966,
        239.        , 229.77380181],
       [ 11.91637529,   0.        ,  11.74734012, ..., 228.01096465,
        239.03765394, 229.70415756],
       [ 10.48808848,  11.74734012,   0.        , ..., 228.08112592,
        238.26665734, 229.77380181],
       ...,
       [228.62851966, 228.01096465, 228.08112592, ...,   0.        ,
        304.13812651, 305.16389039],
       [239.        , 239.03765394, 238.26665734, ..., 304.13812651,
          0.        , 303.10889132],
       [229.77380181, 229.70415756, 229.77380181, ..., 305.16389039,
        303.10889132,   0.        ]])

In [9]:
euclid_dist = pd.DataFrame(dist_distribution,
                           index=customer_matrix.columns, 
                           columns=customer_matrix.columns)

euclid_dist

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.000000,11.916375,10.488088,11.224972,11.401754,11.090537,12.409674,11.045361,11.269428,11.489125,...,206.871941,213.180675,225.656819,198.232187,230.913404,220.501701,217.188858,228.628520,239.000000,229.773802
200,11.916375,0.000000,11.747340,12.083046,12.569805,12.288206,12.165525,12.083046,11.874342,12.000000,...,206.310446,212.635839,224.697575,197.139544,230.952376,220.202180,215.728997,228.010965,239.037654,229.704158
264,10.488088,11.747340,0.000000,11.489125,11.224972,11.445523,12.000000,11.401754,11.180340,11.747340,...,206.387984,212.946003,225.435135,197.600607,230.371439,219.136943,216.612557,228.081126,238.266657,229.773802
356,11.224972,12.083046,11.489125,0.000000,12.083046,11.789826,12.328828,11.135529,11.958261,12.165525,...,206.649462,213.082144,225.452878,197.494304,231.038958,219.952268,217.437347,228.098663,238.493186,229.464594
412,11.401754,12.569805,11.224972,12.083046,0.000000,11.704700,12.328828,11.135529,11.789826,11.747340,...,206.900942,211.679002,225.572605,197.630969,230.614397,219.733930,217.446545,227.997807,238.396728,228.927936
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,220.501701,220.202180,219.136943,219.952268,219.733930,219.599636,219.538152,219.924987,219.827205,220.070443,...,283.945417,283.945417,302.076149,272.717803,278.388218,0.000000,273.861279,291.547595,306.186218,307.205143
98069,217.188858,215.728997,216.612557,217.437347,217.446545,217.425849,216.903204,217.294731,217.080630,216.751009,...,283.945417,283.945417,295.803989,283.945417,285.043856,273.861279,0.000000,287.228132,297.909382,294.745653
98159,228.628520,228.010965,228.081126,228.098663,227.997807,228.197283,228.028507,228.181945,227.868383,228.103047,...,283.945417,279.508497,300.000000,290.473751,300.000000,291.547595,287.228132,0.000000,304.138127,305.163890


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [10]:
euclid_dist.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 33 to 98200
Columns: 1000 entries, 33 to 98200
dtypes: float64(1000)
memory usage: 7.7 MB


In [11]:
# Top 5 most similar customers for CustomerID == 33

top5_similar_33 = euclid_dist[33].sort_values(ascending=False)[1:6]
pd.DataFrame(top5_similar_33)

,33
CustomerID,
97324,247.024290
95017,244.276483
92492,241.642298
91777,240.547293
92525,239.146399


## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [12]:
# Back to Step 1
type(total_quantity)

pandas.core.frame.DataFrame

In [13]:
total_quantity = total_quantity.reset_index()
total_quantity

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1
...,...,...,...
63623,98200,Vol Au Vents,50
63624,98200,Wasabi Powder,25
63625,98200,Wine - Fume Blanc Fetzer,25
63626,98200,Wine - Hardys Bankside Shiraz,25


In [14]:
# Back to Step 4
top5_similar_33 = top5_similar_33.reset_index()
top5_similar_33

,CustomerID,33
0,97324,247.024290
1,95017,244.276483
2,92492,241.642298
3,91777,240.547293
4,92525,239.146399


In [15]:
similar_records = top5_similar_33.merge(total_quantity, on='CustomerID')
similar_records = similar_records.rename(columns={33:'Similarity to ID 33'})
similar_records

,CustomerID,Similarity to ID 33,ProductName,Quantity
0,97324,247.024290,Bagel - Plain,25
1,97324,247.024290,Bandage - Flexible Neon,50
2,97324,247.024290,Bay Leaf,25
3,97324,247.024290,Beef - Inside Round,25
4,97324,247.024290,Beef - Striploin Aa,25
...,...,...,...,...
368,92525,239.146399,Water - Green Tea Refresher,24
369,92525,239.146399,Whmis - Spray Bottle Trigger,24
370,92525,239.146399,Wine - Chablis 2003 Champs,72
371,92525,239.146399,Wine - White Cab Sauv.on,48


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [16]:
# Back to Step Zero
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


In [17]:
# Merging the Top 5 similar customers to the original dataset
purchase_records = similar_records.merge(data, on=['CustomerID','ProductName', 'Quantity'])
columns_order = ['CustomerID','FirstName','LastName','SalesID','ProductID','ProductName','Quantity']
purchase_records = purchase_records[columns_order]
purchase_records

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,97324,Darrick,Williamson,6456451,262,Bagel - Plain,25
1,97324,Darrick,Williamson,844747,202,Bay Leaf,25
2,97324,Darrick,Williamson,1750768,248,Beef - Inside Round,25
3,97324,Darrick,Williamson,5037324,339,Beef - Striploin Aa,25
4,97324,Darrick,Williamson,4117926,14,Beef - Top Sirloin,25
...,...,...,...,...,...,...,...
330,92525,Claude,Dunn,1440045,64,Vinegar - Sherry,24
331,92525,Claude,Dunn,3520876,169,Water - Aquafina Vitamin,24
332,92525,Claude,Dunn,3259791,317,Water - Green Tea Refresher,24
333,92525,Claude,Dunn,6492530,420,Whmis - Spray Bottle Trigger,24


In [18]:
# Ranking

# Ranking
ranking = purchase_records.groupby('ProductName')['Quantity'].sum().reset_index()
ranking = ranking.sort_values(by='Quantity', ascending=False)
ranking = ranking.reset_index(drop=True)
ranking

,ProductName,Quantity
0,"Turkey - Whole, Fresh",74
1,"Coconut - Shredded, Sweet",74
2,Pastry - Raisin Muffin - Mini,74
3,"Bread - Roll, Soft White Round",73
4,Thermometer Digital,73
...,...,...
240,Longos - Grilled Salmon With Bbq,24
241,Milk - 1%,24
242,Muffin - Carrot Individual Wrap,24
243,"Mushroom - Porcini, Dry",24


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [52]:
customer_matrix.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [51]:
# Chosen customer is ID 33 and we are going to filter the unpurchased products
no_purch_list_id33 = customer_matrix[33].reset_index()
no_purch_list_id33 = no_purch_list_id33[no_purch_list_id33[33] == 0]
no_purch_list_id33

,ProductName,33
0,Anchovy Paste - 56 G Tube,0.0
1,"Appetizer - Mini Egg Roll, Shrimp",0.0
2,Appetizer - Mushroom Tart,0.0
3,Appetizer - Sausage Rolls,0.0
5,Apricots - Halves,0.0
...,...,...
446,Wonton Wrappers,0.0
447,Yeast Dry - Fermipan,0.0
448,Yoghurt Tubes,0.0
449,"Yogurt - Blueberry, 175 Gr",0.0


In [53]:
# Merge the ranked products data frame with the customer product matrix on the ProductName field.
recommended_prod = ranking.merge(no_purch_list_id33, on='ProductName')
recommended_prod

,ProductName,Quantity,33
0,"Turkey - Whole, Fresh",74,0.0
1,"Coconut - Shredded, Sweet",74,0.0
2,Pastry - Raisin Muffin - Mini,74,0.0
3,"Bread - Roll, Soft White Round",73,0.0
4,Thermometer Digital,73,0.0
...,...,...,...
213,Longos - Grilled Salmon With Bbq,24,0.0
214,Milk - 1%,24,0.0
215,Muffin - Carrot Individual Wrap,24,0.0
216,"Mushroom - Porcini, Dry",24,0.0


In [54]:
# Show the top 5 results.

recommended_prod.iloc[0:5]

,ProductName,Quantity,33
0,"Turkey - Whole, Fresh",74,0.0
1,"Coconut - Shredded, Sweet",74,0.0
2,Pastry - Raisin Muffin - Mini,74,0.0
3,"Bread - Roll, Soft White Round",73,0.0
4,Thermometer Digital,73,0.0


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [56]:
# Create an empty dictionary that will hold the recommendations for all customers
recommend_for_all = {}

In [59]:
# Create a list of unique CustomerIDs to iterate over.
customers_list = list(data['CustomerID'].unique())
# type(customers_list)
customers_list

[61288,
 77352,
 40094,
 23548,
 78981,
 83106,
 11253,
 35107,
 15088,
 26031,
 86595,
 46715,
 91013,
 69931,
 19753,
 9546,
 10224,
 72846,
 28976,
 3267,
 19260,
 76530,
 30196,
 48818,
 15166,
 11443,
 79060,
 82597,
 62411,
 42611,
 45420,
 83495,
 47122,
 30815,
 53198,
 18591,
 62034,
 1428,
 2187,
 55162,
 22345,
 81876,
 48641,
 81034,
 29681,
 87947,
 28867,
 69967,
 94420,
 83158,
 84364,
 44742,
 7772,
 47303,
 75791,
 77756,
 97324,
 13912,
 52700,
 21734,
 6817,
 21095,
 28875,
 89588,
 1034,
 54827,
 29287,
 11102,
 52155,
 50748,
 88552,
 80938,
 60481,
 92168,
 66232,
 64348,
 37510,
 44408,
 89254,
 477,
 51483,
 97201,
 42157,
 52091,
 86065,
 18027,
 93882,
 42303,
 32010,
 53997,
 63033,
 76475,
 98185,
 84779,
 82146,
 54495,
 3903,
 79854,
 25449,
 85496,
 95017,
 33759,
 43652,
 35873,
 97769,
 891,
 85161,
 25327,
 25624,
 68416,
 44090,
 71595,
 68336,
 38497,
 10016,
 36751,
 58421,
 45498,
 50774,
 79997,
 45313,
 55715,
 40901,
 1577,
 3909,
 18480,
 41405

In [ ]:
# Iterate over the customer list performing steps 4 through 7 for each and appending the results of 
# each iteration to the dictionary you created.

In [76]:
# Lista de diccionarios
recommend_for_all = {}

for customer_dict in customers_list:
    customer_id = customer_dict['CustomerID']
    
    # Step 4: Most similar customers
    most_similar_all = euclid_dist[customer_id].sort_values(ascending=False).reset_index()
    
    # Step 5: Select the records for the list of similar CustomerIDs obtained in previous Step 4 
    similar_records_to_most = most_similar_all.merge(data, on='CustomerID')
    columns_order = ['CustomerID', 'FirstName', 'LastName', 'SalesID', 'ProductID', 'ProductName', 'Quantity']
    similar_records_to_most = similar_records_to_most[columns_order]
    
    # Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field...
    purchase_records = similar_records_to_most.groupby(['CustomerID', 'ProductName'])['Quantity'].sum().reset_index()
    
    # Step 7: Filter the list for products that the chosen customer has not yet purchased and...
    no_purch_list = purchase_records[purchase_records['CustomerID'] == customer_id]
    recommended_prod = no_purch_list.groupby('ProductName')['Quantity'].sum().reset_index()
    recommended_prod = recommended_prod.sort_values(by='Quantity', ascending=False).head(5)
    recommend_for_all[customer_id] = recommended_prod

recommend_for_all_df = pd.concat([pd.DataFrame(recommend_for_all[key]).assign(CustomerID=key) for key in recommend_for_all], ignore_index=True)
recommend_for_all_df = recommend_for_all_df[['CustomerID', 'ProductName', 'Quantity']].rename(columns={'Quantity': 'Recommended_Quantity'})
recommend_for_all_df

IndexError: invalid index to scalar variable.

In [72]:
for customer in customers_list:
    
    # Step 4: Most similar customers
    most_similar_all = euclid_dist[customer].sort_values(ascending=False).reset_index()
    
    # Step 5: Select the records for the list of similar CustomerIDs obtained in previous Step 4 
    similar_records_to_most = most_similar_all.merge(data, on='CustomerID')
    columns_order = ['CustomerID', 'FirstName', 'LastName', 'SalesID', 'ProductID', 'ProductName', 'Quantity']
    similar_records_to_most = similar_records_to_most[columns_order]
    
    # Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field...
    purchase_records = similar_records_to_most.groupby(['CustomerID', 'ProductName'])['Quantity'].sum().reset_index()
    
    # Step 7: Filter the list for products that the chosen customer has not yet purchased and...
    no_purch_list = purchase_records[purchase_records['CustomerID'] == customer]
    recommended_prod = no_purch_list.groupby('ProductName')['Quantity'].sum().reset_index()
    recommended_prod = recommended_prod.sort_values(by='Quantity', ascending=False).head(5)
    recommend_for_all[customer] = recommended_prod
recommend_for_all

{61288:                       ProductName  Quantity
 17           Lamb - Pieces, Diced        48
 41       Wine - White, Mosel Gold        32
 4              Beef - Rib Eye Aaa        32
 5   Beer - Original Organic Lager        32
 37         Wine - Ruffino Chianti        32,
 77352:                        ProductName  Quantity
 79      Wine - Sogrape Mateus Rose        40
 4   Beef - Montreal Smoked Brisket        40
 28  French Pastry - Mini Chocolate        40
 68                    Sugar - Fine        40
 15     Cheese - Brie, Triple Creme        40,
 40094:                  ProductName  Quantity
 12      Cheese - Brie,danish        33
 21             Cumin - Whole        33
 28  Gatorade - Xfactor Berry        22
 23      Curry Paste - Madras        22
 59        Tomatoes Tear Drop        22,
 23548:                          ProductName  Quantity
 41          Scallops - Live In Shell        12
 22         Jolt Cola - Electric Blue        12
 8             Bread - English Muffin  

In [73]:
type(recommend_for_all)

dict

In [71]:
for customer in customers_list:
    
    # Step 4: Most similar customers
    most_similar_all = euclid_dist[customer].sort_values(ascending=False).reset_index()
    
    # Step 5: Select the records for the list of similar CustomerIDs you obtained in previous Step 4 
    similar_records_to_most = most_similar_all.merge(total_quantity, on='CustomerID')
    
    # Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field...
    purchase_records = similar_records_to_most.merge(data, on=['CustomerID','ProductName', 'Quantity'])
    columns_order = ['CustomerID','FirstName','LastName','SalesID','ProductID','ProductName','Quantity']
    purchase_records = purchase_records[columns_order]
    
    # Step 7: Filter the list for products that the chosen customer has not yet purchased and...
    no_purch_list = purchase_records[purchase_records[customer] == 0]
    recommended_prod = no_purch_list.groupby('ProductName')['Quantity'].sum().reset_index()
    recommended_prod = recommended_prod.sort_values(by='Quantity', ascending=False).head(5)
    recommend_for_all[customer] = recommended_prod
recommend_for_all

KeyError: 61288

In [68]:
# Step 4: Most similar customers for CustomerID == 33

# most_similar_33 = euclid_dist[33].sort_values(ascending=False)
# most_similar_33 = most_similar_33.reset_index()
# most_similar_33

In [69]:
# Step 5: Select the records for the list of similar CustomerIDs you obtained in previous Step 4

# similar_records_to_most = most_similar_33.merge(total_quantity, on='CustomerID')
# similar_records_to_most = similar_records_to_most.rename(columns={33:'Similarity to ID 33'})
# similar_records_to_most

In [70]:
# Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field,
# and then rank them in descending order by quantity.

# purchase_records = similar_records_to_most.merge(data, on=['CustomerID','ProductName', 'Quantity'])
# columns_order = ['CustomerID','FirstName','LastName','SalesID','ProductID','ProductName','Quantity']
# purchase_records = purchase_records[columns_order]
# purchase_records

In [ ]:
# Step 7: Filter the list for products that the chosen customer has not yet purchased and 
# then recommend the top 5 products with the highest quantities that are left.



##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.